In [8]:
from metadrive.envs.metadrive_env import MetaDriveEnv
from utils.scenario_runner import ScenarioRunner
from pathlib import Path
import numpy as np

In [9]:
def get_dt(fps):
    # Assume default value
    DECISION_REPEAT = 5
    interval = 1 / fps
    dt = interval / DECISION_REPEAT
    return dt


get_dt(30)

0.006666666666666666

In [10]:
import shutil

SAVE_DIR = Path("/media/olek/2TB_HDD/metadrive-data/sandbox")

# Remove all files in the directory
shutil.rmtree(SAVE_DIR)

SAVE_DIR

PosixPath('/media/olek/2TB_HDD/metadrive-data/sandbox')

In [11]:
seed = 0
FPS = 60

dt = get_dt(FPS)
sr = ScenarioRunner(SAVE_DIR, seed, dt=dt)

In [12]:
config = sr.get_config()
config

{'log_level': 20,
 'start_seed': 0,
 'traffic_density': 0.1,
 'traffic_mode': 'respawn',
 'random_traffic': False,
 'map_config': {'type': 'block_num', 'config': 5},
 'out_of_route_done': False,
 'on_continuous_line_done': False,
 'crash_vehicle_done': False,
 'crash_object_done': False,
 'crash_human_done': False,
 'decision_repeat': 5,
 'physics_world_step_size': 0.003333333333333333}

In [13]:
env = MetaDriveEnv(config)
_, reset_info = env.reset()

[INFO] Environment: MetaDriveEnv
[INFO] MetaDrive version: 0.4.2.3
[INFO] Sensors: [lidar: Lidar(), side_detector: SideDetector(), lane_line_detector: LaneLineDetector()]
[INFO] Render Mode: none
[INFO] Horizon (Max steps per agent): None
[INFO] Assets version: 0.4.2.3
[INFO] Known Pipes: glxGraphicsPipe
[INFO] Start Scenario Index: 0, Num Scenarios : 1


In [ ]:
from metadrive.examples.ppo_expert.numpy_expert import expert

for i in range(500):
    action = expert(env.agent, deterministic=True)
    obs, reward, terminated, truncated, info = env.step(action)

    env.render(
        mode="topdown",
        window=True,
        screen_size=(1000, 1000),
        # show_agent_name=True,
        # semantic_map=True,
        draw_contour=False,
        num_stack=1,
    )


# env.top_down_renderer.generate_gif()
env.close()